In [1]:
import numpy as np
np.random.seed(2016)

import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

from pandas import DataFrame


In [2]:
def get_im_cv2(path):
    img = cv2.imread(path)
    height, width, channels = img.shape
    img = cv2.resize(img, (32, 32))
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#   print height, width, channels
    return img

In [3]:
def load_train():
    X_train = []
    X_train_id = []
    y_train = []
    start_time = time.time()

    print('Read train images')
    folders = ['Automobile',
               'Comics_and_Cartoons',
               'Famous_Personalities',
               'Festivals_and_Occasions',
               'Graffiti_and_Illustrations',
               'Movies_and_TV_shows',
               'Music',
               'Nature',
               'No_Theme',
               'Patterns_and_Ethnic',
               'Signs_and_Symbols',
               'Spiritual',
               'Sports',
               'Superheroes',
               'Typography',
               'Vintage'
              ]
    for fld in folders:
        index = folders.index(fld)
        print('Load folder {} (Index: {})'.format(fld, index))
        path = os.path.join('D:\Misc\CrowdAnalytix\Identifying Themes from Mobile Case Images\Data Raw\New folder\Train', fld, '*.jpg')
        files = glob.glob(path)
        for i in range(len(files)):
            flbase = os.path.basename(files[i])
            img = get_im_cv2(files[i])
            X_train.append(img)
            X_train_id.append(flbase)
            y_train.append(index)   #what does this do?
    print len(X_train)
    print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_train, y_train, X_train_id

In [4]:
def load_test():
    path = os.path.join('D:\Misc\CrowdAnalytix\Identifying Themes from Mobile Case Images\Data Raw\New folder\Test', '*.jpg')
    files = sorted(glob.glob(path))

    X_test = []
    X_test_id = []
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im_cv2(fl)
        X_test.append(img)
        X_test_id.append(flbase)

    return X_test, X_test_id

In [5]:
Xtr, Ytr, Itr = load_train()
X_test, X_test_id = load_test()

Read train images
Load folder Automobile (Index: 0)
Load folder Comics_and_Cartoons (Index: 1)
Load folder Famous_Personalities (Index: 2)
Load folder Festivals_and_Occasions (Index: 3)
Load folder Graffiti_and_Illustrations (Index: 4)
Load folder Movies_and_TV_shows (Index: 5)
Load folder Music (Index: 6)
Load folder Nature (Index: 7)
Load folder No_Theme (Index: 8)
Load folder Patterns_and_Ethnic (Index: 9)
Load folder Signs_and_Symbols (Index: 10)
Load folder Spiritual (Index: 11)
Load folder Sports (Index: 12)
Load folder Superheroes (Index: 13)
Load folder Typography (Index: 14)
Load folder Vintage (Index: 15)
0
Read train data time: 0.02 seconds


In [6]:
# flatten out all images to be one-dimensional
Xtr = np.array(Xtr, dtype=np.uint8)
Ytr = np.array(Ytr, dtype=np.uint8)
msk = np.random.rand(len(Xtr)) < 0.8
Xte = Xtr[~msk]
Yte = Ytr[~msk]
Ytr = Ytr[msk]
Xtr = Xtr[msk]
Xtr_rows = Xtr.reshape(Xtr.shape[0], 32 * 32 * 3) 
Xte_rows = Xte.reshape(Xte.shape[0], 32 * 32 * 3) 
X_test = np.array(X_test, dtype=np.uint8)
X_test_rows = X_test.reshape(X_test.shape[0], 32 * 32 * 3) 

In [10]:
class NearestNeighborL2(object):
  def __init__(self):
    pass

  def train(self, X, y):
    """ X is N x D where each row is an example. Y is 1-dimension of size N """
    # the nearest neighbor classifier simply remembers all the training data
    self.Xtr = X
    self.ytr = y

  def predict(self, X):
    """ X is N x D where each row is an example we wish to predict label for """
    num_test = X.shape[0]
    # lets make sure that the output type matches the input type
    Ypred = np.zeros(num_test, dtype = self.ytr.dtype)

    # loop over all test rows
    for i in xrange(num_test):
      # find the nearest training image to the i'th test image
      # using the L1 distance (sum of absolute value differences)
      distances = np.sqrt(np.sum(np.abs(self.Xtr - X[i,:]), axis = 1))
      min_index = np.argmin(distances) # get the index with smallest distance
      Ypred[i] = self.ytr[min_index] # predict the label of the nearest example

    return Ypred

In [11]:
nn = NearestNeighborL2()
nn.train(Xtr_rows, Ytr)
Yte_predict = nn.predict(Xte_rows)
print 'accuracy: %f' % ( np.mean(Yte_predict == Yte) )

accuracy: 0.489910


In [ ]:
X_test_predict = nn.predict(X_test_rows)
DataFrame(X_test_id,X_test_predict).to_csv("nn_submit_L2.csv")